<a href="https://colab.research.google.com/github/AnjanPayra/Opinion_Voter_Latane/blob/main/LataneModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import necessary libraries**

In [ ]:
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import copy
import math

## **Helper Functions**

A helper function : this is used for visualizing the population of agents.

In [ ]:
def plot_opinions(lattice, title="Opinion Distribution", cmap=None, norm=None):
  """Plot the current state of the lattice with opinion values inside the boxes."""
  fig, ax = plt.subplots(figsize=(8, 8))
  cax = ax.imshow(lattice, cmap=cmap, norm=norm)
  ax.set_xticks(np.arange(-0.5, lattice.shape[1], 1), minor=True)
  ax.set_yticks(np.arange(-0.5, lattice.shape[0], 1), minor=True)
  ax.grid(which="minor", color="black", linestyle="-", linewidth=0.5)
  ax.tick_params(which="both", bottom=False, left=False, labelbottom=False, labelleft=False)

  # Adding a color bar
  plt.colorbar(cax, label="Opinion Value")

  # Annotating each agent with its current opinion
  for i in range(lattice.shape[0]):
    for j in range(lattice.shape[1]):
      ax.text(j, i, str(lattice[i, j]), ha='center', va='center', color='black', fontsize=8)

  plt.title(title)
  plt.show()

In [ ]:
def display_matrix(lattice, title):
  """
  Displays a single lattice with a color gradient, grid lines, and annotations for each value.

  Parameters:
  - lattice: 2D NumPy array representing the lattice to be displayed.
  - title: Optional; title for the plot.
  """
  plt.figure(figsize=(6, 6))
  im = plt.imshow(lattice, cmap='viridis', interpolation='nearest')
  plt.colorbar(label='Value')
  plt.title(title)

  # Add grid lines
  plt.grid(color='black', linestyle='-', linewidth=0.5)
  plt.xticks(np.arange(-0.5, lattice.shape[1], 1), [])
  plt.yticks(np.arange(-0.5, lattice.shape[0], 1), [])
  plt.gca().set_xticks(np.arange(-0.5, lattice.shape[1], 1), minor=True)
  plt.gca().set_yticks(np.arange(-0.5, lattice.shape[0], 1), minor=True)
  plt.grid(which="minor", color="black", linestyle='-', linewidth=0.5)

  # Annotate each cell with its value
  for i in range(lattice.shape[0]):
      for j in range(lattice.shape[1]):
          value = lattice[i, j]
          plt.text(j, i, f'{value:.2f}', ha='center', va='center', color='white', fontsize=8)

  plt.show()

## **Defining the lattice.**

In [ ]:
def initialize_lattice(L):
  values = np.arange(1, L * L + 1)
  lattice = values.reshape((L, L))
  sval = np.random.rand(L, L)
  pval = np.random.rand(L, L)
  #sval = 1 - pval
  return lattice, sval, pval

## **Latane Sim**

In [ ]:
def chooseNextColour(klist):
  """Choose the next colour based on the probabilities."""
  #generate a random number between 0 and 1
  random_number = random.random()  # Generate a random number between 0 and 1
  cumulative_probability = 0
  for i, probability in enumerate(klist):
    cumulative_probability += probability
    if random_number <= cumulative_probability:
      return i+1
  # If the loop completes without finding a class, return the last class
  # This can happen due to floating-point errors
  #return len(probabilities)

In [ ]:
def Kdelta(a, b):
  """Return the Kronecker delta of two values i and j."""
  return 1 if a == b else 0

In [ ]:
def func_g(coord1, coord2, alpha):
  # Compute the Euclidean distance
  dist = np.linalg.norm(np.array(coord1) - np.array(coord2))
  # Scale the distance
  scaled_dist = 1 + dist**alpha
  return scaled_dist

In [ ]:
def get_prob(val, T):
  if val ==0:
    return 0
  else:
    return math.exp(val/T)

In [ ]:
def normalize_list(I_klist):
  total_sum = sum(I_klist)
  if total_sum == 0:  # Handle case where sum is zero to avoid division by zero
    return [0] * len(I_klist)
  else:
    normalized_list = [element / total_sum for element in I_klist]
    return normalized_list

In [ ]:
def get_influence(lattice, xi, yi, k, s, p, alpha):
  """Update the opinion of neighbors based on the Sznajd model for a 2D lattice."""
  L = lattice.shape[0]
  I = 0
  for xj in range(L):
    for yj in range(L):
      sj = s[xj, yj]
      pj = p[xj, yj]
      if lattice[xi, yi] == lattice[xj, yj]:
        print(f"point i = {lattice[xi, yi]}; j = {lattice[xj, yj]} ; k = {k}; del(k,j)={Kdelta(k, lattice[xj, yj])} ; del(i,j)={Kdelta(lattice[xj, yj], lattice[xi, yi])} ; Support")
        I = I + ((4*sj)/func_g((xi, yi),(xj, yj), alpha))*Kdelta(k, lattice[xj, yj])*Kdelta(lattice[xj, yj], lattice[xi, yi])
      else:
        print(f"point i = {lattice[xi, yi]}; j = {lattice[xj, yj]} ; k = {k}; del(k,j)={Kdelta(k, lattice[xj, yj])} ; del(i,j)={Kdelta(lattice[xj, yj], lattice[xi, yi])} ; Perssuade")
        I = I + ((4*pj)/func_g((xi, yi),(xj, yj), alpha))*Kdelta(k, lattice[xj, yj])*(1-Kdelta(lattice[xj, yj], lattice[xi, yi]))
  return I

In [ ]:
def lataneSimD(lattice, steps, stepSize_, s, p, alpha, temp, cmap, norm):
  """Run the Latane model simulation on a lattice for a given number of steps (Deterministic)"""
  countOpinions = []
  unique_elements_count = len(np.unique(lattice))
  countOpinions.append(unique_elements_count)
  L = lattice.shape[0]
  for run in range(steps):
    colours = np.unique(lattice)
    print(len(colours))
    lattice_new = copy.deepcopy(lattice)
    for xi in range(L):
      for yi in range(L):
        max_I = -1
        klist = []
        for k in colours:
          I_ijk = get_influence(lattice, xi, yi, k, s, p, alpha)
          klist.append(I_ijk)
          #print("\n I_ijk = ",I_ijk)
          if I_ijk > max_I:
            max_I = I_ijk
            max_k = k
        #print(klist)
        lattice_new[xi, yi] = max_k
    lattice = copy.deepcopy(lattice_new)
    unique_elements_count = len(np.unique(lattice))
    countOpinions.append(unique_elements_count)
    if (run+1) % stepSize_ == 0:
      plot_opinions(lattice, title=f"Epoch {run+1}", cmap=cmap, norm=norm)
    if unique_elements_count == 1:
      print(f"Converged after {run+1} epochs. with count {unique_elements_count}")
      plot_opinions(lattice, title="Final Opinion Distribution")
      return countOpinions

In [ ]:
def lataneSimP(lattice, steps, stepSize_, s, p, alpha, temp, cmap, norm):
  """Run the Latane model simulation on a lattice for a given number of steps (Probabilistic)"""
  countOpinions = []
  unique_elements_count = len(np.unique(lattice))
  countOpinions.append(unique_elements_count)
  L = lattice.shape[0]
  for run in range(steps):
    colours = np.unique(lattice)
    print(len(colours))
    lattice_new = copy.deepcopy(lattice)
    for xi in range(L):
      for yi in range(L):
        I_klist = []
        print(colours)
        for k in colours:
          I_ijk = get_influence(lattice, xi, yi, k, s, p, alpha)
          prob_I = get_prob(I_ijk, temp)
          I_klist.append(prob_I)
        I_nlist = normalize_list(I_klist)
        lattice_new[xi, yi] = chooseNextColour(I_nlist)
    print("\n Old: \n",lattice,"\n New: \n",lattice_new)
    lattice = copy.deepcopy(lattice_new)
    unique_elements_count = len(np.unique(lattice))
    countOpinions.append(unique_elements_count)
    if (run+1) % stepSize_ == 0:
      plot_opinions(lattice, title=f"Epoch {run+1}", cmap=cmap, norm=norm)
    if unique_elements_count == 1:
      print(f"Converged after {run+1} epochs. with count {unique_elements_count}")
      plot_opinions(lattice, title="Final Opinion Distribution")
      return countOpinions

Use the function lataneSimD for deterministic version and lataneSimP for probablistic version.

In [ ]:
# Initialize and simulate
L = 3 # length of side of the lattice
epochs = 100000 # number of epochs (optional)
stepSize = 1
max_opinion = L * L // 2
#max_opinion = 3
alpha = 2
temp = 1.5
norm = mcolors.Normalize(vmin=1, vmax=max_opinion)
cmap = plt.cm.viridis  # Use the viridis colormap, or change to another continuous colormap

lattice, sval, pval = initialize_lattice(L)
#lattice = np.array([[1, 2, 3], [2, 2, 2], [1, 1, 1]])
#print(lattice)
#sval = np.array([[0.7, 0.4, 0.1], [0.8, 0.5, 0.2], [0.9, 0.6, 0.3]])
#pval = np.array([[0.3, 0.6, 0.9], [0.2, 0.5, 0.8], [0.1, 0.4, 0.7]])
#print(type(lattice))
colours = np.unique(lattice)
print(len(colours))
plot_opinions(lattice, title="Initial Opinion Distribution", cmap=cmap, norm=norm)
display_matrix(sval, title="Initial s-value Distribution")
display_matrix(pval, title="Initial p-value Distribution")

countOpinions =  lataneSimP(lattice, epochs, stepSize, sval, pval, alpha, temp, cmap=cmap, norm=norm) # for probablistic version
#countOpinions =  lataneSimD(lattice, epochs, stepSize, sval, pval, alpha, temp, cmap=cmap, norm=norm) # for deterministic version

In [ ]:
len(countOpinions)

24

In [ ]:
countOpinions[-10:]

[23, 23, 22, 22, 20, 20, 18, 13, 5, 1]

In [ ]:
epochs_range = range(len(countOpinions))

plt.plot(epochs_range, countOpinions)
plt.xlabel("Epochs")
plt.ylabel("Number Opinions")
plt.title("Opinions vs. Epochs")
plt.grid(True)
plt.show()

In [ ]:
epochs_range = range(len(countOpinions))

plt.figure(figsize=(10, 6))  # Adjust figure size if needed
plt.scatter(epochs_range, countOpinions, marker='o', s=10)  # Scatter plot
plt.xlabel("Epochs")
plt.ylabel("Number of Opinions")
plt.title(f"Number of Opinions vs. Epochs (Log Scale) L= {L}")
plt.grid(True)

# Set the x and y axes to log scale
plt.xscale("log")
plt.yscale("log")

plt.show()